In [18]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0,'../')
sys.path.insert(0,'../../../data_analysis/')
from simulate_door_slam import *
from trajectory import *
from utils import *
from local_plot import *

def read_frame_timestamps(g2o_path, nodes):
    frame_id_to_stamp = {}
    for i in nodes:
        with open(f"{g2o_path}/frame_timestamp_{i}.txt", "r") as f:
            for line in f:
                frame_id, stamp = line.split()
                frame_id_to_stamp[int(frame_id)] = float(stamp)
    return frame_id_to_stamp

def process_output(g2o_path, nodes):
    poses_pgo, t0 = read_paths(g2o_path, nodes, prefix="pgo_")
    paths_gt, _ = read_paths(g2o_path + "/../", nodes, prefix="groundtruth_", t0=t0)
    align_paths(poses_pgo, paths_gt, align_by_first=True, align_with_minize=True)
    display(plot_fused_err(nodes, poses_pgo, paths_gt, show=False))
    # plot_fused(nodes, poses_pgo, paths_gt)
    # display(relative_pose_err(nodes, poses_pgo, paths_gt, dte=dte))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Note
We need to start ROS Core Manually for this file

In [47]:
g2o_path="/home/xuhao/data/d2slam/pgo/tum_corr_5/"
g2o_input = g2o_path + "/input/"
max_steps = 100
eta_k=1.4545454545454546
rho_frame_T=0.39
rho_frame_theta=1.5555885588558855
simulate_delay_ms=0.0
enable_linear_pose6d_solver = False
pg = PoseGraph(g2o_path + "/raw/")
print(f"Initial cost: {pg.evaluate_cost()} edges {len(pg.edges)}")
pg.write_to_g2o_folder(g2o_input, cvt_id=True)
agent_num = 5
frame_id_to_stamp = read_frame_timestamps(g2o_path, range(agent_num))
output_path=g2o_path + "/d2pgo-rot-inited/"
pgo_optimized, max_solve_time, min_solve_time = call_d2pgo_opti(g2o_folder=g2o_input, output_folder=output_path, enable_rot_init=True, 
    max_steps=max_steps, agent_num=agent_num, ignore_infor=False, eta_k=eta_k, rho_frame_theta=rho_frame_theta, 
    rho_frame_T=rho_frame_T, simulate_delay_ms=simulate_delay_ms, enable_linear_pose6d_solver=enable_linear_pose6d_solver)
pgo_optimized.write_to_csv(output_path, frame_id_to_stamp)
print(f"D2PGO RotInit cost: {pgo_optimized.evaluate_cost()} solve_time {max_solve_time:.1f}ms")
process_output(output_path, [0, 1, 2, 3, 4])

output_path=g2o_path + "/d2pgo-no-rot-init/"
pgo_optimized, max_solve_time, min_solve_time = call_d2pgo_opti(g2o_folder=g2o_input, output_folder=output_path, enable_rot_init=False, 
    max_steps=max_steps, agent_num=agent_num, ignore_infor=False, eta_k=eta_k, rho_frame_theta=rho_frame_theta, 
    rho_frame_T=rho_frame_T, simulate_delay_ms=simulate_delay_ms)
pgo_optimized.write_to_csv(output_path, frame_id_to_stamp)
print(f"D2PGO without RotInit cost: {pgo_optimized.evaluate_cost()} solve_time {max_solve_time:.1f}ms")
process_output(output_path, [0, 1, 2, 3, 4])

output_path = g2o_path + "/DGS/"
iterations, min_time, max_time, initial, final, util_rate, total_optim = call_DGS_solver(g2o_input, agent_num=agent_num, 
    rthresh=1e-4, pthresh=1e-4, maxIter=50, between_noise="true")
pgo_optimized = copy.copy(pg)
pg.read_g2o_single(g2o_input+"/fullGraph_optimized.g2o", update_only=True)
pg.write_to_csv(output_path, frame_id_to_stamp)
total_time = max_time + iterations*100 #Assume communication time is 100ms
print(f"cost: {pgo_optimized.evaluate_cost()} max_time: {max_time} iter {iterations} total_time {total_time} DGS cost initial {initial} final: {final} ")
process_output(output_path, [0, 1, 2, 3, 4])


Total agents 5 keyframes 12065 edges 16814 inter edge 3165 comm_vol 4754 keyframes 759<->3030
Initial cost: 350408.55991278985 edges 16814
Total agents 5 keyframes 12065 edges 16814 inter edge 3165 comm_vol 4754 keyframes 759<->3030
D2PGO RotInit cost: 61.514716967546775 solve_time 12741.3ms


Drone,Traj. Len.,ATE Pos,ATE Att,Cov/m: x,y,z,Cov Att/m
0,295.2m,0.656,3.767,2.2e-02,6.9e-02,1.2e-02,3.88e-02
1,308.5m,0.592,3.571,1.1e-04,1.3e-04,6.9e-04,4.48e-03
2,297.8m,0.605,3.517,1.4e-04,8.8e-05,5.1e-04,5.12e-03
3,110.2m,0.515,3.079,3.0e-04,4.4e-04,1.4e-03,3.04e-03
4,265.2m,0.880,4.657,2.6e-04,5.5e-05,4.0e-04,6.95e-03
Avg.,255.4,0.650,3.72,,,,


Total agents 5 keyframes 12065 edges 16814 inter edge 3165 comm_vol 4754 keyframes 759<->3030
D2PGO without RotInit cost: 60409.287983741866 solve_time 9202.5ms


Drone,Traj. Len.,ATE Pos,ATE Att,Cov/m: x,y,z,Cov Att/m
0,299.3m,1.407,4.920,3.0e-03,1.9e-02,4.5e-04,4.73e-03
1,308.7m,1.295,5.472,1.3e-04,7.0e-04,5.3e-04,3.34e-03
2,314.7m,1.302,8.640,6.5e-04,1.9e-03,3.4e-04,6.86e-03
3,114.2m,1.106,2.467,1.1e-03,1.1e-02,8.0e-04,6.82e-03
4,264.8m,1.740,4.626,5.5e-04,7.3e-04,3.3e-04,2.57e-03
Avg.,260.3,1.370,5.22,,,,


cost: 58.98641475974949 max_time: 1809.3 iter 58 total_time 7609.3 DGS cost initial 236205.1 final: 72.798 


Drone,Traj. Len.,ATE Pos,ATE Att,Cov/m: x,y,z,Cov Att/m
0,295.6m,0.039,1.782,2.8e-04,1.7e-04,5.0e-04,3.13e-03
1,308.7m,0.043,0.825,1.1e-04,1.2e-04,3.9e-04,5.47e-03
2,298.1m,0.043,1.200,7.8e-05,1.0e-04,5.7e-04,3.71e-03
3,110.4m,0.042,0.811,1.1e-04,3.0e-04,1.0e-03,1.22e-03
4,265.3m,0.043,0.781,2.6e-04,8.5e-05,3.4e-04,6.37e-03
Avg.,255.6,0.042,1.08,,,,
